In [13]:
import json
import pandas as pd

with open(r"C:\Users\mvanysac\OneDrive - MINFIN\Bureaublad\wielermanager\Tour_2025.json", "r") as f:
    data = json.load(f)

df_players = pd.json_normalize(data["players"]) #we pakken enkel deel onder players

df_active_players = df_players[df_players["active"] == 1] #enkel deelnemende renners

df_value = pd.DataFrame(df_active_players[["name", "value"]]) #renners en hun wielermanagerwaarde

#data zoals onedayraces enzo zijn colommen geworden met "speciality...."
speciality_columns = [col for col in df_players.columns if col.startswith('speciality.')]

df_speciality_expanded = df_players[speciality_columns].copy()


df_speciality_expanded.columns = [col.replace('speciality.', '') for col in df_speciality_expanded.columns]

df_speciality = pd.concat([df_active_players[['name', "value"]], df_speciality_expanded], axis=1) #renners en hun pro cycling stats
df_speciality.reset_index(drop=True, inplace=True)
df_speciality.index = df_speciality.index + 1
df_speciality.head()

,name,value,Onedayraces,GC,TT,Sprint,Climber,Hills
1,Pogacar Tadej,13.0,7677.0,6844.0,2703.0,302.0,8498.0,3706.0
2,Jonas Vingegaard,11.0,320.0,5026.0,2064.0,129.0,4390.0,1561.0
3,Evenepoel Remco,10.0,3370.0,4254.0,4131.0,120.0,4068.0,1544.0
4,Philipsen Jasper,9.0,5089.0,574.0,154.0,5820.0,185.0,2358.0
5,Kooij Olav,8.0,1712.0,585.0,72.0,2922.0,48.0,942.0


In [14]:
df_gt_all_normalized  = pd.json_normalize(data['players'], record_path=['GT'], meta=['id','name', 'value', 'active'], errors='ignore')

df_recent_gt_active_players = df_gt_all_normalized[(df_gt_all_normalized['active'] == 1) & (df_gt_all_normalized["seasonId"] >= 2024)].copy()

df_gt_final = df_recent_gt_active_players[['name', 'value', "seasonId",'competitionFeed', 'GC', 'points', 'youth', 'mountain', 'best']].copy()
df_gt_final['GC'] = df_gt_final['GC'].replace(0, 999).fillna(999)
df_gt_final[['points', 'youth', 'mountain', 'best']] = df_gt_final[['points', 'youth', 'mountain', 'best']].fillna(0)

#gewogen gemiddeldes pakken waarbij tour harder doortelt
df_gt_final['gewicht'] = 1 # 1 als standaard
df_gt_final.loc[df_gt_final['competitionFeed'] == 'TOUR', 'gewicht'] = 2 # Tour telt 2x zo zwaar



def weighted_mean(rank, weights):
    return (rank * weights).sum() / weights.sum()



df_gt_tour_multiple_agg = df_gt_final.groupby("name").agg(
    gc_average=('GC', lambda x: weighted_mean(x, df_gt_final.loc[x.index, 'gewicht'])),
    groen_average=('points', lambda x: weighted_mean(x, df_gt_final.loc[x.index, 'gewicht'])),
    best_result=('best', 'min'), 
    count=('seasonId', 'count')
)

for col in df_gt_tour_multiple_agg:
    df_gt_tour_multiple_agg[col] = df_gt_tour_multiple_agg[col].round(0).astype(int)
    

df_gt_tour_multiple_agg = df_gt_tour_multiple_agg.sort_values(by=["gc_average", "best_result"])

df_gt_tour_multiple_agg.head()


,gc_average,groen_average,best_result,count
name,,,,
Pogacar Tadej,1,4,1,2
Jonas Vingegaard,2,10,1,1
Evenepoel Remco,3,7,1,1
O'connor Ben,3,32,1,2
Skjelmose Mattias,5,17,3,1


In [15]:
df_seasonStats_normalized  = pd.json_normalize(data['players'], record_path=['seasonStats'], meta=['id','name', 'value', 'active'], errors='ignore')

# enkel seizoen 2025
df_2025_active_players = df_seasonStats_normalized[(df_seasonStats_normalized['active'] == 1) & (df_seasonStats_normalized["season"] == 2025)].copy()

df_season2025 = df_2025_active_players[['name', 'value', "season",'racedays', 'wins', 'top3s', 'top10s']].copy()

df_season2025[['wins', 'top3s', 'top10s']] = df_season2025[['wins', 'top3s', 'top10s']].fillna(0)

df_season2025.sort_values(by=["wins", "top3s", "top10s"], ascending=False, inplace=True)
df_season2025.reset_index(drop=True, inplace=True)
df_season2025.index = df_season2025.index + 1

df_season2025.head()

,name,value,season,racedays,wins,top3s,top10s
1,Pogacar Tadej,13,2025,22,11,16,18
2,Matthew Brennan,5,2025,29,10,14,14
3,Merlier Tim,8,2025,29,9,12,12
4,Pedersen Mads,8,2025,41,8,12,21
5,Almeida Joao,7,2025,35,6,16,21


In [ ]:
#alles naar excel op verschillende werkbaden

excel_bestandsnaam = 'TOUR2025.xlsx'
writer = pd.ExcelWriter(excel_bestandsnaam, engine='xlsxwriter')
df_speciality.to_excel(writer, sheet_name='Procyclingstats', index=False)
df_gt_tour_multiple_agg.to_excel(writer, sheet_name='GT_average', index=False)
df_season2025.to_excel(writer, sheet_name='Season2025', index=False)

print(f"Alle DataFrames zijn succesvol geëxporteerd naar '{excel_bestandsnaam}' met verschillende werkbladen.")
writer.save()
print("Excel-bestand is opgeslagen.")